In [63]:
# ==============================================
# 0) Package installation
# ==============================================
!pip install -q --upgrade pip
!pip install -q kaggle beautifulsoup4 lxml pandas tqdm scikit-learn python-dotenv pillow pytesseract pdf2image google-genai
!apt-get update -qq && apt-get install -y -qq tesseract-ocr poppler-utils


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [64]:
# ==============================================
# 1) Imports & configuration
# ==============================================
import os, re, zipfile, math
from pathlib import Path
from getpass import getpass
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from PIL import Image
from pdf2image import convert_from_path
from google import genai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
from google.colab import userdata
import time

# ------------------------------
# GEMINI API key
# ------------------------------
api_key = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=api_key)


In [65]:
# ==============================================
# 2) Upload & extract dataset ZIP
# ==============================================
from google.colab import files
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
EXTRACT_DIR = Path("financial_dataset")
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(EXTRACT_DIR)
print("Extracted to:", EXTRACT_DIR)

Saving archive.zip to archive.zip
Extracted to: financial_dataset


In [66]:
# ==============================================
# 3) HTML -> text parser
# ==============================================
def html_to_text(html_content: str) -> str:
    soup = BeautifulSoup(html_content, "lxml")
    for s in soup(["script", "style"]):
        s.extract()
    text = soup.get_text(separator="\n")
    return re.sub(r"\n\s*\n+", "\n\n", text).strip()

In [67]:
# ==============================================
# 4) OCR functions for images & PDFs
# ==============================================
import pytesseract

def ocr_image_file(path: Path) -> str:
    img = Image.open(path)
    return pytesseract.image_to_string(img, lang='eng')

def ocr_pdf_file(path: Path) -> str:
    pages = convert_from_path(str(path), dpi=200, first_page=1, last_page=3)
    return "\n\n".join(pytesseract.image_to_string(p, lang='eng') for p in pages)

In [68]:
# ==============================================
# 5) Parse all files and build dataset
# ==============================================
rows = []

html_files = list(EXTRACT_DIR.rglob("*.html"))
pdf_files = list(EXTRACT_DIR.rglob("*.pdf"))
image_files = []
for ext in ["png","jpg","jpeg","tiff","bmp"]:
    image_files += list(EXTRACT_DIR.rglob(f"*.{ext}"))

def extract_basic_fields(text: str):
    date_match = re.search(r"Date[: ]+([0-9]{4}-[0-9]{2}-[0-9]{2}|[0-9]{2}/[0-9]{2}/[0-9]{4}|[0-9]{1,2} [A-Za-z]+ [0-9]{4})", text)
    total_match = re.search(r"(Total|TOTAL|Amount)[: ]+([0-9,.]+)", text)
    supplier_match = re.search(r"(Supplier|Vendor|From)[: ]+(.+)", text)
    return {
        "date": date_match.group(1) if date_match else None,
        "total": float(total_match.group(2).replace(",","")) if total_match else None,
        "supplier": supplier_match.group(2).strip() if supplier_match else None
    }

# process HTML
for f in tqdm(html_files, desc="HTML files"):
    text = html_to_text(f.read_text(encoding='utf-8', errors='ignore'))
    rows.append({
        "file": str(f),
        "text": text,
        "source_type": "html",
        **extract_basic_fields(text)
    })

# process PDFs
for f in tqdm(pdf_files, desc="PDF files"):
    try:
        text = ocr_pdf_file(f)
        rows.append({
            "file": str(f),
            "text": text,
            "source_type": "pdf",
            **extract_basic_fields(text)
        })
    except Exception as e:
        print("PDF OCR error:", f, e)

# process images
for f in tqdm(image_files, desc="Image files"):
    try:
        text = ocr_image_file(f)
        rows.append({
            "file": str(f),
            "text": text,
            "source_type": "image",
            **extract_basic_fields(text)
        })
    except Exception as e:
        print("Image OCR error:", f, e)

df = pd.DataFrame(rows)
print("Dataset rows:", len(df))

HTML files: 100%|██████████| 2525/2525 [00:34<00:00, 72.84it/s]
PDF files: 0it [00:00, ?it/s]
Image files: 0it [00:00, ?it/s]

Dataset rows: 2525


In [69]:
# ==============================================
# 6) Gemini few-shot classification
# ==============================================
FEW_SHOT_EXAMPLES = [
    ("Balance Sheet example: Assets 100000 Liabilities 40000", "Balance Sheet"),
    ("Cash Flow Statement showing cash from operations: 5000", "Cash Flow"),
    ("Income Statement: Revenue 150000 Net Income 12000", "Income Statement"),
    ("Notes: accounting policies and footnotes for financial statements", "Notes"),
    ("Invoice for office supplies purchased", "Others"),
]

def classify_with_gemini_text(text, max_input_chars=3800):
    shots = "\n\n".join([f"Document: {s}\nLabel: {l}" for s,l in FEW_SHOT_EXAMPLES])
    sample = text[:max_input_chars].replace("\n"," ")
    prompt = (
        f"{shots}\n\nInstruction: Return exactly one label (case-sensitive): "
        "Balance Sheet, Cash Flow, Income Statement, Notes, Others.\n\n"
        f"Document: {sample}\nLabel:"
    )
    resp = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)
    out = resp.text.strip()
    for label in ["Balance Sheet","Cash Flow","Income Statement","Notes","Others"]:
        if label.lower() in out.lower(): return label
    return out.splitlines()[0].strip().split()[0][:40]

#df['gemini_label'] = [classify_with_gemini_text(t) for t in tqdm(df['text'], desc="Gemini classification")]

# 0️⃣ Nustatymai
TOTAL_DOCS = 20      # kiek dokumentų iš viso paimti
BATCH_SIZE = 10       # kiek vienu metu siųsti į API
SLEEP_TIME = 60      # pauzė po batch (sekundėmis)

# 1️⃣ Paimti atsitiktinę dalį dokumentų
sample_df = df.sample(TOTAL_DOCS, random_state=42).reset_index(drop=True)
texts = sample_df['text'].tolist()

# 2️⃣ Batch klasifikacija su pauze
labels = []
for start_idx in range(0, TOTAL_DOCS, BATCH_SIZE):
    end_idx = min(start_idx + BATCH_SIZE, TOTAL_DOCS)
    batch_texts = texts[start_idx:end_idx]

    for text in tqdm(batch_texts, desc=f"Gemini classification {start_idx}-{end_idx-1}"):
        try:
            label = classify_with_gemini_text(text)
        except Exception as e:
            print("Gemini error:", e)
            label = "ERROR"
        labels.append(label)

    if end_idx < TOTAL_DOCS:
        print(f"Batch done ({end_idx}/{TOTAL_DOCS}). Waiting {SLEEP_TIME} sec to respect API limits...")
        for remaining in range(SLEEP_TIME, 0, -1):
            print(f"\rCountdown: {remaining:2d} sec ", end="")
            time.sleep(1)
        print("\rCountdown:  0 sec - resuming next batch!        ")

# 3️⃣ Priskirti rezultatus atgal į DataFrame
sample_df['gemini_label'] = labels
print(sample_df[['file','gemini_label']])

Gemini classification 0-9: 100%|██████████| 10/10 [00:28<00:00,  2.87s/it]


Batch done (10/20). Waiting 60 sec to respect API limits...
Countdown:  0 sec - resuming next batch!        


Gemini classification 10-19: 100%|██████████| 10/10 [00:22<00:00,  2.26s/it]

                                                 file      gemini_label
0      financial_dataset/data/Others/18929978_83.html             Notes
1   financial_dataset/data/Notes/18950920_table_10...             Notes
2   financial_dataset/data/Balance Sheets/18950392...     Balance Sheet
3   financial_dataset/data/Notes/18731570_table_73...  Income Statement
4      financial_dataset/data/Others/18776682_19.html            Others
5   financial_dataset/data/Balance Sheets/18917547...     Balance Sheet
6       financial_dataset/data/Others/18931924_7.html  Income Statement
7   financial_dataset/data/Notes/18646837_table_93...             Notes
8      financial_dataset/data/Others/18628308_12.html             Notes
9   financial_dataset/data/Income Statement/184753...  Income Statement
10     financial_dataset/data/Others/18586111_13.html            Others
11      financial_dataset/data/Others/18540991_4.html             Notes
12  financial_dataset/data/Notes/18888243_table_14...  Income St

In [70]:
# ==============================================
# 7) TF-IDF + RandomForest classifier (ground truth from folder names)
# ==============================================
def infer_label_from_path(pathstr):
    low = pathstr.lower()
    if "balance" in low: return "Balance Sheet"
    if "cash" in low: return "Cash Flow"
    if "income" in low: return "Income Statement"
    if "notes" in low: return "Notes"
    return "Others"

df['gt_label'] = df['file'].apply(infer_label_from_path)

texts = df['text'].fillna("").tolist()
y = df['gt_label'].tolist()
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1,2))
X = vectorizer.fit_transform(texts)

rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X, y, cv=cv, scoring='f1_macro')
print("RandomForest 5-fold F1-macro:", scores.mean())

rf.fit(X, y)
df['rf_label'] = rf.predict(vectorizer.transform(df['text'].fillna("").tolist()))

RandomForest 5-fold F1-macro: 0.9422604559145956


In [71]:
# ==============================================
# 8) Map to accounting entries (suderinta su batch Gemini)
# ==============================================
def map_to_accounting(label, text, total=None):
    if label == "Income Statement": debit, credit = "7000 Revenue", "4430 Accounts Payable"
    elif label == "Balance Sheet": debit, credit = "1000 Assets", "2000 Equity/Liabilities"
    elif label == "Cash Flow": debit, credit = "1500 Cash", "6000 Cash Flow Adjustment"
    elif label == "Notes": debit, credit = "9999 Notes", "0000"
    else: debit, credit = "6300 General Expense", "4430 Accounts Payable"

    if total is not None: amt = total
    else:
        m = re.search(r"(Total|AMOUNT|Amount)[: ]+([0-9,.]+)", text, re.IGNORECASE)
        amt = float(m.group(2).replace(",","")) if m else None
    return {"debit": debit, "credit": credit, "amount": amt}

entries = []
for idx, row in df.iterrows():
    # Jei 'gemini_label' egzistuoja (subset), naudoti jį; jei ne – rf_label
    chosen_label = row['gemini_label'] if 'gemini_label' in row and row['gemini_label'] not in ["ERROR", None] else row['rf_label']

    entry = map_to_accounting(chosen_label, row['text'], total=row.get('total', None))

    entries.append({
        "file": row['file'],
        "chosen_label": chosen_label,
        "debit": entry['debit'],
        "credit": entry['credit'],
        "amount": entry['amount']
    })

entries_df = pd.DataFrame(entries)
entries_df.head(20)

,file,chosen_label,debit,credit,amount
0,financial_dataset/data/Cash Flow/18630222_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
1,financial_dataset/data/Cash Flow/18779279_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
2,financial_dataset/data/Cash Flow/19058356_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
3,financial_dataset/data/Cash Flow/18646837_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
4,financial_dataset/data/Cash Flow/18661965_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
5,financial_dataset/data/Cash Flow/18630222_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
6,financial_dataset/data/Cash Flow/19442329_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
7,financial_dataset/data/Cash Flow/18630219_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
8,financial_dataset/data/Cash Flow/18661963_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
9,financial_dataset/data/Cash Flow/19070958_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN


In [72]:
# ==============================================
# 9) Save outputs
# ==============================================
df.to_csv("dataset_with_labels.csv", index=False)
entries_df.to_csv("results.csv", index=False)
print("Saved: dataset_with_labels.csv and results.csv")

# Preview
entries_df.head(20)

Saved: dataset_with_labels.csv and results.csv


,file,chosen_label,debit,credit,amount
0,financial_dataset/data/Cash Flow/18630222_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
1,financial_dataset/data/Cash Flow/18779279_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
2,financial_dataset/data/Cash Flow/19058356_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
3,financial_dataset/data/Cash Flow/18646837_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
4,financial_dataset/data/Cash Flow/18661965_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
5,financial_dataset/data/Cash Flow/18630222_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
6,financial_dataset/data/Cash Flow/19442329_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
7,financial_dataset/data/Cash Flow/18630219_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
8,financial_dataset/data/Cash Flow/18661963_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
9,financial_dataset/data/Cash Flow/19070958_tabl...,Cash Flow,1500 Cash,6000 Cash Flow Adjustment,NaN
